# Scoreboard Class

In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

sys.path.insert(0, '../YTCollab/src')

import extract_data
import scoreboard

First, before we can actually utilize the class, we must have a DataFrame of channel data, and the data of an individual channel. For this demonstration, we'll use my own channel as the individual channel, and as my channel is of the Gaming category, we'll score it agains the Gaming data set that's been extracted previously.

In [2]:
lichwick = 'UCcx01rLXJ4dCtXqW3kPmBzg'

df = pd.read_csv('../YTCollab/data/YTGaming_data.csv').drop('Unnamed: 0', axis=1)
y = extract_data.get_specific_channel(lichwick)

../YTCollab/src/extract_data.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_new[topic][i] = count


In [3]:
df.head()

,id,title,desc,country,banner,subs,videos,views,/m/01yjl,/m/02hrh1q,...,/m/057qgw,/m/06wfzt,/m/07f_7h,/m/012dwj48,/m/01ct6,/m/079m5l,/m/043mm3,/m/029l86,/m/04sv4,/m/02zw41
0,UC0M0rxSz3IF0CsSour1iWmw,Cinemassacre,Cinemassacre Productions creates many differen...,n/a,https://yt3.ggpht.com/-eJAg-IPdtsQ/VfdBt5u90gI...,2029622,758,830331820,0,0,...,0,0,0,0,0,0,0,0,0,0
1,UCxaVOVnhmT0-HCUv72jtOTA,FaZe Rain,Welcome to my Channel!\n\nBe sure to connect w...,CA,https://yt3.ggpht.com/-TYWqsuLUMTM/VpgeFGfscOI...,3218809,1306,512123287,0,0,...,0,0,0,0,0,0,0,0,0,0
2,UCC-RHF_77zQdKcA75hr5oTQ,FGTeeV,"CHANNEL DESCRIPTION\nFGTeeV, the Family Gaming...",US,https://yt3.ggpht.com/-ajrs7jpr9w4/Vp-rVhuffVI...,1537023,890,1685682328,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UCYVinkwSX7szARULgYpvhLw,Ali-A,Ali-A // Subscribe and join the #AliAarmy!\n\n...,GB,https://yt3.ggpht.com/-XHud3sgrMts/VuFT6ZtpmaI...,7985200,2117,1916404064,0,0,...,0,0,0,0,0,0,0,0,0,0
4,UCzYfz8uibvnB7Yc1LjePi4g,Aphmau,☆ Evening ladies and gents! Welcome to my Yout...,US,https://yt3.ggpht.com/-w87jwkRlZFI/U_gZFaAtNXI...,1655338,1836,591661288,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y

,id,title,desc,country,banner,subs,videos,views,/m/09g9srb,/m/013f7bmb,...,/m/0282f,/m/06zm8z,/m/02x9hs,/m/0403l3g,/m/0y7v_dk,/m/03qh21m,/m/03bt1gh,/m/0y6cg_2,/m/0p8xwrr,/m/0215n
0,UCcx01rLXJ4dCtXqW3kPmBzg,LichWick,"Hello, I'm LichWick. I like playing games and ...",US,https://yt3.ggpht.com/-jLrCPhtaQT8/VWD27D3emBI...,64,187,5057,2,2,...,2,7,2,15,2,2,26,9,2,2


Note that our data set contains 7276 features while our individual channel only contains 28. As long as the two DataFrames follow a specific structure, the Scoreboard accounts for this difference of features. Luckily, the extract_data module will always output DataFrames in the format the Scoreboard works from so we don't need to worry about this as long as we're using data pulled with an extract_data function.

Scoreboard has a similar initialization style as most models available in sklearn. The object can be created, and then fitted with our data. The large data set must be specified first, and our individual data second. From there, Scoreboard scales and organizes the data, scoring it by cosine_similarity and ordering it by similarity score from highest to lowest.

The output is optimized for web integration with flask, but we can still retrieve what we need from the results. The ouput is a list of dictionaries, each with a title entry specifying the name of the channel in that position.

In [5]:
sim = scoreboard.Scoreboard()
sim.fit(df,y)
results = sim.most_similar()
result_titles = [channel['title'] for channel in results]
result_titles

/home/alex/anaconda2/lib/python2.7/site-packages/pandas-0.17.1-py2.7-linux-x86_64.egg/pandas/core/ops.py:648: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  result = lib.scalar_compare(x, y, op)


['BedBananas',
 "Gamer's Little Playground",
 'Sips',
 'Lobos Jr',
 'bisnap',
 'SplatterCatGaming',
 'DidYouKnowGaming?',
 'RaedwulfGamer',
 'SLEEPCYCLES',
 'Northernlion']

So, for my channel, the top 3 channels (ie: the 3 most similar channels) are BedBananas, Gamer's Little Playground, and Sips, in that order. By default, Scoreboards most_similar function returns the top 10 results, but the placements the user cares about can be specified. For instance, if one said most_similar(5), they would receive the top 5 results instead of the top 10. Alternatively, most_similar(10,20) would return the results that ranked from 10 to 20, maintaining their order in terms of score.

The web interface integrating the Scoreboard class requests most_similar(25), for example.